<a href="https://colab.research.google.com/github/quaziharis/machine_Learning/blob/master/Bengaluru_house_price_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pycaret

     |████████████████████████████████| 194kB 4.4MB/s 
     |████████████████████████████████| 7.0MB 662kB/s 
     |████████████████████████████████| 1.6MB 53.8MB/s 
     |████████████████████████████████| 389kB 47.5MB/s 
     |████████████████████████████████| 266kB 49.3MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 63.9MB 65kB/s 
     |████████████████████████████████| 1.2MB 41.4MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 3.0MB 37.7MB/s 
     |████████████████████████████████| 133kB 49.1MB/s 
     |████████████████████████████████| 552kB 41.4MB/s 
     |████████████████████████████████| 2.1MB 41.8MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 235kB 48.3MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 6.1MB 40.5MB/s 
   

In [0]:
import pandas as pd
df=pd.read_csv("/content/Bengaluru_House_Data.csv")
df.head(10)

,area_type,availability,location,size,BHK,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2.0,BHK,Coomee,1056.0,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4.0,Bedroom,Theanmp,2600.0,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3.0,BHK,NaN,1440.0,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3.0,BHK,Soiewre,1521.0,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2.0,BHK,NaN,1200.0,2.0,1.0,51.00
5,Super built-up Area,Ready To Move,Whitefield,2.0,BHK,DuenaTa,1170.0,2.0,1.0,38.00
6,Super built-up Area,18-May,Old Airport Road,4.0,BHK,Jaades,2732.0,4.0,NaN,204.00
7,Super built-up Area,Ready To Move,Rajaji Nagar,4.0,BHK,Brway G,3300.0,4.0,NaN,600.00
8,Super built-up Area,Ready To Move,Marathahalli,3.0,BHK,NaN,1310.0,3.0,1.0,63.25
9,Plot Area,Ready To Move,Gandhi Bazar,6.0,Bedroom,NaN,1020.0,6.0,NaN,370.00


In [0]:
df.shape

(13320, 10)

In [0]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train = df.sample(frac=0.8, random_state=786).reset_index(drop=True)
test = df.drop(train.index).reset_index(drop=True)

print('Data for Modeling: ' + str(train.shape))
print('Unseen Data For Predictions ' + str(test.shape))

Data for Modeling: (10656, 10)
Unseen Data For Predictions (2664, 10)


In [0]:
train=train.dropna()

In [0]:
from pycaret.regression import *
exp_reg101 = setup(data = train, target = 'price', session_id=123,
                  normalize = True, transformation = True,transform_target = True,
                  remove_multicollinearity = False, 
                  high_cardinality_features=['location','society'],
                  feature_selection=True,
                  combine_rare_levels = True,
                  ignore_low_variance=True, 
                  polynomial_features=True,
                  polynomial_degree=3,
                  ignore_features=['bath','BHK','availability'],
                  remove_outliers=True,outliers_threshold=0.05) 

 
Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Transform Target,True
2,Transform Target Method,box-cox
3,Original Data,"(5881, 10)"
4,Missing Values,False
5,Numeric Features,4
6,Categorical Features,5
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,frequency


In [0]:
compare_models(blacklist=['ransac','tr','kr','mlp'],fold=10)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,16.934200,1449.753600,36.220700,0.779800,0.246400,0.181000
1,Light Gradient Boosting Machine,17.223800,1515.444700,37.067000,0.771400,0.247500,0.182300
2,Random Forest,17.386000,1553.869800,37.788600,0.760400,0.261800,0.187400
3,Extreme Gradient Boosting,19.137500,1693.640800,39.459000,0.739200,0.265000,0.203000
4,Gradient Boosting Regressor,19.085500,1708.803500,39.722100,0.737100,0.264500,0.202400
5,Extra Trees Regressor,17.937900,1703.205100,39.694000,0.731200,0.278300,0.198600
6,Support Vector Machine,20.488400,1947.949500,42.841400,0.694300,0.289900,0.222700
7,Ridge Regression,21.312400,1984.612600,43.203200,0.687700,0.295700,0.233300
8,Bayesian Ridge,21.314800,1983.860100,43.197300,0.687700,0.295700,0.233300
9,Linear Regression,21.310400,1985.318700,43.208700,0.687600,0.295700,0.233300


In [0]:
random_forest=create_model('rf',fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.9993,872.4713,29.5376,0.7407,0.2705,0.1889
1,17.5615,1118.2543,33.4403,0.8244,0.2288,0.1665
2,20.3416,2689.2009,51.8575,0.6473,0.3102,0.1907
3,17.1142,927.3209,30.4519,0.8212,0.2632,0.1951
4,15.5627,868.0991,29.4635,0.8371,0.2740,0.2180
5,17.6001,1890.0259,43.4744,0.7930,0.2418,0.1789
6,17.6315,1349.5685,36.7365,0.7272,0.2557,0.1838
7,21.4302,4054.2896,63.6733,0.6108,0.2904,0.1956
8,16.6753,1042.1245,32.2820,0.7878,0.2507,0.1804
9,13.9435,727.3431,26.9693,0.8149,0.2329,0.1758


In [0]:
random_forest

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)

In [0]:
tuned_rf=tune_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.7659,843.8965,29.0499,0.7492,0.2653,0.1887
1,17.5887,1148.6462,33.8917,0.8196,0.2234,0.1638
2,20.2126,2541.9999,50.4182,0.6666,0.3020,0.1915
3,17.0959,936.3328,30.5996,0.8195,0.2591,0.1942
4,15.8660,900.0956,30.0016,0.8311,0.2767,0.2214
5,17.5752,1967.4290,44.3557,0.7846,0.2408,0.1778
6,17.2589,1141.3820,33.7843,0.7692,0.2430,0.1777
7,21.6545,3994.9181,63.2054,0.6165,0.2873,0.1959
8,16.8627,1063.0356,32.6042,0.7835,0.2523,0.1830
9,14.4991,785.2713,28.0227,0.8001,0.2391,0.1816


In [0]:
tuned_rf

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=80, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=80, n_jobs=None, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)

In [0]:
cat_boost=create_model('catboost',fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.3587,754.5886,27.4698,0.7757,0.2516,0.1816
1,16.3904,960.7680,30.9963,0.8491,0.2085,0.1584
2,18.9446,2274.8069,47.6949,0.7017,0.2837,0.1794
3,16.6270,959.3906,30.9740,0.8150,0.2485,0.1878
4,16.8858,1231.1468,35.0877,0.7690,0.2624,0.2135
5,16.5263,1506.8872,38.8186,0.8350,0.2236,0.1682
6,15.9992,975.1350,31.2272,0.8029,0.2260,0.1672
7,22.5708,4396.1469,66.3034,0.5780,0.2892,0.1981
8,15.6589,739.4914,27.1936,0.8494,0.2435,0.1803
9,14.3804,699.1747,26.4419,0.8220,0.2269,0.1761


In [0]:
tuned_catboost=tune_model('catboost',n_iter=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.1039,760.7727,27.5821,0.7739,0.2513,0.1791
1,16.3259,983.5107,31.3610,0.8456,0.2075,0.1564
2,18.7295,2197.1889,46.8742,0.7119,0.2817,0.1797
3,16.7474,1061.9192,32.5871,0.7953,0.2509,0.1871
4,16.9000,1209.9581,34.7845,0.7730,0.2643,0.2166
5,17.0170,1822.3987,42.6896,0.8005,0.2279,0.1723
6,15.8665,963.8320,31.0456,0.8051,0.2270,0.1674
7,22.7606,4464.0388,66.8135,0.5715,0.2914,0.1988
8,15.4669,667.1444,25.8291,0.8642,0.2426,0.1800
9,14.4865,737.8043,27.1626,0.8122,0.2287,0.1777


In [0]:
light=create_model('lightgbm',fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.4440,768.3644,27.7194,0.7716,0.2502,0.1802
1,17.3679,1008.8751,31.7628,0.8416,0.2122,0.1595
2,18.8158,2155.4093,46.4264,0.7173,0.2791,0.1789
3,17.0783,1126.5493,33.5641,0.7828,0.2513,0.1871
4,16.4396,1157.4056,34.0207,0.7828,0.2655,0.2154
5,17.7887,2000.3521,44.7253,0.7810,0.2279,0.1748
6,16.5158,1048.5490,32.3813,0.7880,0.2299,0.1720
7,22.5709,4470.1015,66.8588,0.5709,0.2893,0.1987
8,15.7567,775.8652,27.8544,0.8420,0.2422,0.1794
9,14.4603,642.9753,25.3570,0.8363,0.2271,0.1767


In [0]:
tuned_lgbm=tune_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,14.6658,712.7712,26.6978,0.7881,0.2562,0.1756
1,17.3274,1183.2641,34.3986,0.8142,0.2224,0.1622
2,19.2889,2315.3368,48.1179,0.6964,0.2886,0.1800
3,16.4413,882.6507,29.7094,0.8298,0.2488,0.1848
4,15.1149,961.5491,31.0089,0.8196,0.2553,0.1992
5,17.0318,1693.7398,41.1551,0.8145,0.2321,0.1767
6,16.3410,1067.1947,32.6679,0.7842,0.2353,0.1715
7,21.9899,4336.0941,65.8490,0.5838,0.2949,0.1922
8,15.3454,740.5700,27.2134,0.8492,0.2375,0.1729
9,13.5709,604.6464,24.5896,0.8461,0.2269,0.1727


In [0]:
bagged_tune_lgbm = ensemble_model(tuned_lgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,14.3735,691.5639,26.2976,0.7944,0.2498,0.1727
1,16.5531,987.3853,31.4227,0.8449,0.2147,0.1569
2,18.8477,2191.0508,46.8087,0.7127,0.2888,0.1811
3,16.5120,973.0128,31.1932,0.8124,0.2482,0.1845
4,15.0154,914.9996,30.2490,0.8283,0.2553,0.1997
5,16.4699,1499.1929,38.7194,0.8358,0.2258,0.1698
6,16.9009,1154.5176,33.9782,0.7666,0.2407,0.1772
7,21.3872,4288.3173,65.4852,0.5884,0.2891,0.1896
8,15.5261,807.5801,28.4180,0.8356,0.2426,0.1747
9,14.0259,627.0395,25.0408,0.8404,0.2265,0.1741


In [0]:
bagged_lgbm = ensemble_model(clf3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.3529,729.5888,27.0109,0.7831,0.2500,0.1818
1,16.8384,931.7553,30.5247,0.8537,0.2102,0.1579
2,19.1353,2194.4750,46.8452,0.7122,0.2843,0.1844
3,17.1376,1152.3899,33.9469,0.7778,0.2522,0.1878
4,16.1092,1063.2284,32.6072,0.8005,0.2644,0.2154
5,17.3115,1813.7045,42.5876,0.8014,0.2281,0.1713
6,17.5222,1232.9040,35.1127,0.7507,0.2343,0.1778
7,22.6646,4504.3504,67.1145,0.5676,0.2929,0.2010
8,16.2385,870.4845,29.5040,0.8227,0.2461,0.1825
9,15.0874,728.0951,26.9832,0.8147,0.2327,0.1829


In [0]:
bagged_tune_catboost = ensemble_model(tuned_catboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.4737,761.7814,27.6004,0.7736,0.2539,0.1851
1,16.9767,990.8883,31.4784,0.8444,0.2127,0.1608
2,19.0551,2195.5446,46.8566,0.7121,0.2850,0.1854
3,17.2462,1118.1068,33.4381,0.7844,0.2555,0.1921
4,16.9000,1228.5138,35.0502,0.7695,0.2664,0.2170
5,17.3326,1791.1596,42.3221,0.8039,0.2303,0.1747
6,16.4587,988.4159,31.4391,0.8002,0.2303,0.1725
7,22.8979,4517.6441,67.2134,0.5664,0.2935,0.2012
8,16.2111,765.2542,27.6632,0.8442,0.2482,0.1853
9,15.3514,799.6376,28.2779,0.7965,0.2338,0.1839


In [0]:
bagged_tune_rf = ensemble_model(tuned_rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16.4073,859.1545,29.3113,0.7446,0.2629,0.1938
1,17.8482,1203.3995,34.6900,0.8110,0.2238,0.1629
2,20.2722,2411.5411,49.1074,0.6837,0.2999,0.1969
3,17.2066,1006.1598,31.7200,0.8060,0.2586,0.1925
4,16.6083,1112.9742,33.3613,0.7912,0.2798,0.2258
5,17.7966,1990.7519,44.6178,0.7820,0.2412,0.1790
6,18.2731,1325.4925,36.4073,0.7320,0.2449,0.1830
7,22.1823,4237.9901,65.0998,0.5932,0.2906,0.2002
8,17.6486,1156.8486,34.0125,0.7644,0.2594,0.1914
9,15.5049,838.4462,28.9559,0.7866,0.2415,0.1864


In [0]:
boosted_lgb = ensemble_model(light, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16.3492,906.5965,30.1097,0.7305,0.2624,0.1861
1,18.2473,1124.4561,33.5329,0.8234,0.2209,0.1689
2,19.8450,2307.6324,48.0378,0.6974,0.2876,0.1862
3,17.8209,1111.1686,33.3342,0.7858,0.2553,0.1961
4,16.9775,1120.1945,33.4693,0.7898,0.2654,0.2181
5,18.7884,2215.1860,47.0658,0.7574,0.2389,0.1851
6,18.0491,1293.4018,35.9639,0.7385,0.2481,0.1902
7,23.4710,4495.2424,67.0466,0.5685,0.2937,0.2051
8,16.4841,823.0604,28.6890,0.8324,0.2456,0.1858
9,15.1972,683.4768,26.1434,0.8260,0.2329,0.1844


In [0]:
blend_all = blend_models()

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,18.6341,1020.5372,31.9458,0.6967,0.2821,0.2172
1,22.9973,2067.1373,45.4658,0.6754,0.2615,0.1913
2,22.9163,3199.8813,56.5675,0.5804,0.3124,0.2091
3,19.7402,1653.3468,40.6614,0.6812,0.2753,0.2136
4,20.6611,1931.5849,43.9498,0.6376,0.3079,0.2587
5,23.0132,3847.9596,62.0319,0.5787,0.2777,0.2135
6,19.5314,1737.1070,41.6786,0.6488,0.2640,0.1936
7,26.9666,6067.2729,77.8927,0.4176,0.3299,0.2294
8,20.6499,1601.8301,40.0229,0.6738,0.2846,0.2151
9,17.7795,1138.4750,33.7413,0.7102,0.2609,0.2120


In [0]:
clf1 = create_model('rf', verbose = False)
clf2 = create_model('xgboost', verbose = False)
clf3 = create_model('lightgbm', verbose = False)
clf4 = create_model('catboost', verbose = False)
blend_specific = blend_models(estimator_list = [clf1,clf2,clf3,clf4],fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.6470,765.5582,27.6687,0.7725,0.2521,0.1839
1,16.8455,985.8025,31.3975,0.8452,0.2082,0.1577
2,19.2420,2333.3199,48.3045,0.6940,0.2860,0.1812
3,16.8504,1013.4874,31.8353,0.8046,0.2498,0.1871
4,16.3756,1072.0682,32.7425,0.7989,0.2649,0.2166
5,17.1361,1746.7131,41.7937,0.8087,0.2256,0.1708
6,16.8009,1096.5442,33.1141,0.7783,0.2308,0.1708
7,22.4730,4389.6560,66.2545,0.5786,0.2868,0.1973
8,16.1022,805.5221,28.3817,0.8360,0.2426,0.1800
9,14.2241,686.1596,26.1946,0.8254,0.2266,0.1753


In [0]:
clf2=create_model('rf',verbose=False)
clf3 = create_model('lightgbm', verbose = False)
clf4 = create_model('catboost', verbose = False)
blend_specific = blend_models(estimator_list = [clf2,clf3,clf4],fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.2798,757.8180,27.5285,0.7748,0.2513,0.1795
1,16.4140,955.5754,30.9124,0.8499,0.2075,0.1551
2,18.8372,2313.8433,48.1024,0.6966,0.2854,0.1769
3,16.4441,956.9625,30.9348,0.8155,0.2485,0.1841
4,15.9099,1026.2670,32.0354,0.8074,0.2617,0.2108
5,16.6708,1742.3646,41.7416,0.8092,0.2242,0.1674
6,16.2878,1034.0198,32.1562,0.7909,0.2298,0.1683
7,21.8409,4271.0335,65.3531,0.5900,0.2843,0.1925
8,15.7375,789.2220,28.0931,0.8393,0.2399,0.1763
9,13.8255,645.6196,25.4090,0.8357,0.2225,0.1705


In [0]:
stack_1 = stack_models([clf2,clf3,clf4])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.1213,746.8524,27.3286,0.7780,0.2502,0.1779
1,16.3554,972.5159,31.1852,0.8473,0.2080,0.1552
2,18.8906,2306.1029,48.0219,0.6976,0.2841,0.1767
3,16.4002,972.9053,31.1914,0.8124,0.2475,0.1834
4,16.1740,1098.1419,33.1382,0.7940,0.2621,0.2115
5,16.4795,1617.6597,40.2201,0.8229,0.2236,0.1676
6,15.9113,980.3295,31.3102,0.8018,0.2268,0.1659
7,22.1266,4382.7850,66.2026,0.5793,0.2867,0.1932
8,15.3666,711.2825,26.6699,0.8552,0.2411,0.1776
9,13.9230,660.5230,25.7006,0.8319,0.2231,0.1718


In [0]:
stack_2 = stack_models([clf2,clf3,clf4],meta_model=random_forest)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.4683,845.7526,29.0818,0.7486,0.2575,0.1802
1,17.3282,1124.5166,33.5338,0.8234,0.2166,0.1594
2,19.5138,2399.1431,48.9810,0.6854,0.2904,0.1813
3,16.7717,1034.7571,32.1676,0.8005,0.2594,0.1885
4,16.2502,1131.6565,33.6401,0.7877,0.2668,0.2143
5,16.1723,1867.2394,43.2116,0.7955,0.2235,0.1625
6,16.0451,1006.7554,31.7294,0.7965,0.2298,0.1684
7,22.9056,4759.3567,68.9881,0.5432,0.2989,0.1983
8,15.8302,840.7990,28.9965,0.8288,0.2441,0.1757
9,14.2575,712.2892,26.6887,0.8187,0.2252,0.1700


In [0]:
stack_3 = stack_models([clf2,clf3,clf4,bagged_tune_lgbm,tuned_lgbm],restack=False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,14.5091,692.9874,26.3247,0.7940,0.2483,0.1735
1,16.3170,1024.0677,32.0011,0.8392,0.2075,0.1547
2,18.5012,2226.1644,47.1822,0.7081,0.2802,0.1731
3,15.8644,872.4914,29.5380,0.8318,0.2419,0.1791
4,15.8145,1079.2645,32.8522,0.7975,0.2542,0.2038
5,16.2547,1501.0035,38.7428,0.8356,0.2205,0.1662
6,15.6505,964.5280,31.0569,0.8050,0.2239,0.1629
7,22.1623,4373.1598,66.1299,0.5802,0.2879,0.1923
8,14.8956,675.5091,25.9906,0.8625,0.2349,0.1715
9,13.5807,610.0503,24.6992,0.8447,0.2208,0.1692


In [0]:
xgboost=create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,17.2359,859.2413,29.3128,0.7446,0.2666,0.2032
1,18.7358,1163.3348,34.1077,0.8173,0.2249,0.1746
2,21.1283,2491.7626,49.9176,0.6732,0.2987,0.2019
3,18.6711,1274.5660,35.7011,0.7543,0.2641,0.2058
4,18.3258,1279.9287,35.7761,0.7599,0.2852,0.2423
5,19.2219,1849.8248,43.0096,0.7974,0.2453,0.1940
6,18.9354,1383.5894,37.1966,0.7203,0.2486,0.1880
7,24.8720,4828.9003,69.4903,0.5365,0.3042,0.2205
8,17.9439,933.7247,30.5569,0.8099,0.2626,0.2015
9,16.3054,871.5355,29.5218,0.7782,0.2494,0.1988


In [0]:
stacknet = create_stacknet([[clf2,clf3,bagged_tune_lgbm,tuned_lgbm], [ cat_boost]])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,14.6856,711.3245,26.6707,0.7886,0.2491,0.1738
1,16.8595,1022.2642,31.9729,0.8395,0.2123,0.1565
2,18.6279,2190.9212,46.8073,0.7127,0.2814,0.1759
3,16.3612,974.4144,31.2156,0.8121,0.2462,0.1822
4,15.7481,1031.2916,32.1137,0.8065,0.2588,0.2084
5,17.1023,1826.1690,42.7337,0.8000,0.2260,0.1726
6,16.2577,1052.3881,32.4405,0.7872,0.2291,0.1689
7,22.2629,4467.6354,66.8404,0.5712,0.2901,0.1932
8,15.2820,742.4915,27.2487,0.8488,0.2374,0.1734
9,13.7256,588.4067,24.2571,0.8502,0.2235,0.1718


In [0]:
predict_model(stack_3);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,16.1366,893.6806,29.8945,0.7983,0.2391,0.1792


In [0]:
predict_model(stacknet);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,16.2991,937.7047,30.622,0.7884,0.241,0.181


In [0]:
final_stack_3 = finalize_model(stack_3)

In [0]:
unseen_predictions = predict_model(final_stack_3, data=test, round=0)
unseen_predictions.head(20)

,area_type,availability,location,size,BHK,society,total_sqft,bath,balcony,price,Label
0,Super built-up Area,Ready To Move,Chamrajpet,2.0,BHK,NaN,650.0,1.0,1.0,40.00,27.0
1,Super built-up Area,Ready To Move,7th Phase JP Nagar,3.0,BHK,SrncyRe,1370.0,2.0,1.0,54.79,63.0
2,Super built-up Area,Ready To Move,Whitefield,3.0,BHK,AjhalNa,1725.0,3.0,2.0,106.00,106.0
3,Built-up Area,Ready To Move,Jalahalli,2.0,BHK,NaN,1000.0,2.0,0.0,70.00,45.0
4,Plot Area,Ready To Move,TC Palaya,1.0,Bedroom,NaN,1350.0,1.0,0.0,55.00,69.0
5,Plot Area,Ready To Move,7th Phase JP Nagar,4.0,Bedroom,NaN,3200.0,4.0,2.0,350.00,254.0
6,Super built-up Area,Ready To Move,Bellandur,3.0,BHK,NaN,1717.0,3.0,2.0,110.00,103.0
7,Super built-up Area,21-Dec,Kanakpura Road,2.0,BHK,PrarePa,700.0,2.0,1.0,34.00,36.0
8,Super built-up Area,Ready To Move,Shanthi Pura,2.0,BHK,ShtonDw,800.0,2.0,1.0,32.00,32.0
9,Super built-up Area,Ready To Move,Rajaji Nagar,3.0,BHK,PhestOn,2367.0,4.0,3.0,340.00,357.0
